In [1]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import glob
import os
import pandas as pd

class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


In [3]:
file_list = glob.glob('raw_data/*')
file_list.sort()


In [6]:
bar = ProgBar(len(file_list),"Reading and appending files...")

converted_files = glob.glob('pickle/*')

for file in file_list:

    file_name = file.split('/')[-1]

    convert_name = "pickle/" + file_name + ".pkl"

    if convert_name not in converted_files:

        data = pd.read_csv(file)

        pickle.dump(data.values,
                    open(convert_name, "wb"), 
                    )
                    
    bar.update()
    

Reading and appending files...
[--------------------------------------------------] 100.00% - 16 of 16

In [2]:
converted_files = glob.glob('pickle/*')

converted_files.sort()

In [6]:
bar = ProgBar(len(converted_files),"Reading data files...")

for f,file in enumerate(converted_files):

    if f == 0:

        data  = pickle.load(open(file,'rb'))
        
        aux = [data[0,0],1,data[0,2],data[0,3],data[0,4],data[0,5],data[0,6],data[0,7]]

        data = np.vstack((aux,data))
    
        target = np.zeros(len(data)).reshape(-1,1)

        data = np.hstack((data,target))


    else:

        new_data = pickle.load(open(file,'rb'))

        aux = [new_data[0,0],1,new_data[0,2],new_data[0,3],new_data[0,4],new_data[0,5],new_data[0,6],new_data[0,7]]

        new_data = np.vstack((aux,new_data))

        if f < 11:
            target = np.zeros(len(new_data)).reshape(-1,1)

        elif f >=11 and f < 13:
            target = np.ones(len(new_data)).reshape(-1,1)

        else:
            target = np.ones(len(new_data)).reshape(-1,1)

            target[:] = 2

        new_data = np.hstack((new_data,target))

        data = np.vstack((data,new_data))    

        bar.update()

n_measures = int(np.max(data[:, 1]))

n_ids = len(np.unique(data[:,0]))

Reading data files...
[                                                  ] 0.00% - 0 of 16(4641280, 9)
[----------------------------------------         ] 81.25% - 13 of 16

In [5]:

percentages = [33]

count = 0

bar = ProgBar(len(converted_files)*33,"Performing data scaling...")

divisions = glob.glob('pickle/*')

for i in range(33):

    for f,file in enumerate(converted_files):

        data  = pickle.load(open(file,'rb'))

        aux = [data[0,0],1,data[0,2],data[0,3],data[0,4],data[0,5],data[0,6],data[0,7]]

        data = np.vstack((aux,data))

        if f < 11:
            target = np.zeros(len(data)).reshape(-1,1)

            data = np.hstack((data,target))

        elif f >=11 and f < 13:
            target = np.ones(len(data)).reshape(-1,1)

            data = np.hstack((data,target))

        else:
            target = np.ones(len(data)).reshape(-1,1)

            target[:] = 2

            data = np.hstack((data,target))

        n_measures = int(np.max(data[:, 1]))
        n_ids = len(np.unique(data[:,0]))

        expanded_dimensions = data.reshape(n_ids, n_measures, 9)

        _, fold = train_test_split(expanded_dimensions,
                                    test_size=0.33)

        train_fold, test_fold = train_test_split(fold,
                                                    test_size=0.3)

        train_fold = train_fold.reshape(int(len(train_fold)*n_measures),9)
        test_fold = test_fold.reshape(int(len(test_fold)*n_measures),9)

        if f == 0:
            train_division = train_fold
            test_division = test_fold

        else:
            train_division = np.vstack((train_division,train_fold))
            test_division = np.vstack((test_division,test_fold))

        bar.update()


    pickle.dump(train_division,
                open("/media/thiago/b69516cd-ab99-487f-9f04-062f0eaea877/train_divisions/division__{}__{}__.pkl".format(count,i), "wb"))

    pickle.dump(test_division,
                open("/media/thiago/b69516cd-ab99-487f-9f04-062f0eaea877/test_divisions/division__{}__{}__.pkl".format(count,i), "wb"))

    count += 1



Performing data scaling...
[--------------------------------------------------] 100.00% - 528 of 528

In [11]:
test = pickle.load(open('train_divisions/division__0__0__.pkl','rb'))

df = pd.DataFrame(test)

df.head()

,0,1,2,3,4,5,6,7,8
0,927.0,1.0,2.414551,2.334595,2.441406,3.433838,3.161926,3.263245,0.0
1,927.0,2.0,2.313843,3.615723,3.267822,2.175903,2.526245,2.271729,0.0
2,927.0,3.0,2.443237,2.385559,2.391052,1.618652,1.683655,1.683655,0.0
3,927.0,4.0,2.131958,3.139038,1.729431,2.413635,2.177429,2.881470,0.0
4,927.0,5.0,2.639465,2.279053,2.498474,3.016357,3.011475,3.064270,0.0


In [9]:
len(np.unique(train_fold[:,0]))

23